In [ ]:
import pandas as pd 
import numpy as np 
import time

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet152

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
labels = pd.read_csv('../input/appa-real-face-cropped/labels.csv')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                  validation_split=0.2, 
                                  horizontal_flip=True)

train_datagen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='../input/appa-real-face-cropped/final_files/final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=12345)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.2)
test_datagen_flow = test_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='../input/appa-real-face-cropped/final_files/final_files',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=12345)

In [ ]:
optimizer = Adam(lr = 0.0001)
backbone = ResNet152(input_shape=(224, 224, 3),
                        weights='imagenet', 
                        include_top=False)

model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [ ]:
my_callbacks = [
    EarlyStopping(patience=10),
    ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5')
]

In [ ]:
def train_model(model, train_data, test_data, batch_size=None, steps_per_epoch=None, validation_steps=None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=40,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2, shuffle=True,
              callbacks=my_callbacks)

    return model

In [ ]:
%%time
trained_model = train_model(model, train_datagen_flow, test_datagen_flow)